In [1]:
import datetime as dt
import json
import time
from enum import Enum, auto
from pathlib import Path

import numpy as np
import nbtlib
import mcworldlib as mc

In [2]:
PATH_DREHMAL_SAVE = Path(r"C:\Users\Zachary\AppData\Roaming\.minecraft_drehmal\saves\Drehmal APOTHEOSIS 2.2.0")
PATH_DREHMAL_level_dat = Path(r"C:\Users\Zachary\AppData\Roaming\.minecraft_drehmal\saves\Drehmal APOTHEOSIS 2.2.0\level.dat")

SAVE_DIRECTORY = Path("../data_raw/").resolve()

class Dimension(Enum):
    Overworld = 1
    End = 2
    Space = 3
    Lodahr = 4
    TrueEnd = 5

class MajorItem(Enum):
    Default = 0
    Chest = 1
    Barrel = 2
    Shulker = 3
    Lectern = 4
    Artifact = auto()
    Artisan = auto()
    Trinket = auto()
    Legendary = auto()
    Mythical = auto()
    # Items
    DivineBauble = auto()
    RunicCatalyst = auto()
    RunicAmplifier = auto()
    RelicVessel = auto()
    ScrollOfSanctuary = auto()  # Trinket
    # Special
    StoneAgony = auto()
    StoneLuxury = auto()
    StoneWorry = auto()
    # Lore (written_book, writable_book, some paper)
    LoreBook = auto()
    # Apples
    GayApple = auto()
    GodApple = auto()
    # Other
    # Nihilist Notes
    # Compass of Nihility
    # Snake Ornamented Compass
    # Lotus Shaped Compass
    # Netherite items (3)
    WitherSkull = auto()

DIMENSION = {Dimension.Overworld: "region/",
             Dimension.End: "DIM1/region/",
             Dimension.Space: "dimensions/minecraft/space/",
             Dimension.Lodahr: "dimensions/minecraft/lodahr/",
             Dimension.TrueEnd: "dimensions/minecraft/true_end/"}

# Item frames are entities
TILE_ENTITIES_ITEMS = {"minecraft:chest", "minecraft:trapped_chest",
                      "minecraft:barrel",
                      "minecraft:brewing_stand",
                      "minecraft:smoker",
                      "minecraft:dispenser",
                      "minecraft:dropper",
                      "minecraft:furnace",
                      "minecraft:blast_furnace",
                      "minecraft:hopper",
                      "minecraft:shulker_box",
                      "minecraft:white_shulker_box",
                      "minecraft:orange_shulker_box",
                      "minecraft:magenta_shulker_box",
                      "minecraft:light_blue_shulker_box",
                      "minecraft:yellow_shulker_box",
                      "minecraft:lime_shulker_box",
                      "minecraft:pink_shulker_box",
                      "minecraft:gray_shulker_box",
                      "minecraft:light_gray_shulker_box",
                      "minecraft:cyan_shulker_box",
                      "minecraft:purple_shulker_box",
                      "minecraft:blue_shulker_box",
                      "minecraft:brown_shulker_box",
                      "minecraft:green_shulker_box",
                      "minecraft:red_shulker_box",
                      "minecraft:black_shulker_box"}
TILE_ENTITIES_BOOK = {"minecraft:lectern"}
TILE_ENTITIES_OTHER = {"minecraft:wither_skeleton_skull",
                       "minecraft:wither_skeleton_wall_skull",
                       "minecraft:ender_chest"}
KEEP_TILE_ENTITIES = TILE_ENTITIES_ITEMS | TILE_ENTITIES_BOOK | TILE_ENTITIES_ITEMS

In [11]:
# Coordinate ranges
COORDINATES = {
    "Overworld": {"x_min": -5632,
                  "x_max": 7167,
                  "z_min": -5120,
                  "z_max": 7679}
}

In [3]:
print(PATH_DREHMAL_level_dat.exists())

True


# Setup

In [7]:
world = mc.load(PATH_DREHMAL_level_dat)
mc.pretty(world.regions)
regions = world.regions[mc.OVERWORLD]
print("Count of regions:", len(regions))
entities = world.entities[mc.OVERWORLD]
# # print(regions[0, 0])
# print(type(regions), type(entities))
# print(len(regions), len(entities))  # count of regions in a world
# region = regions[0, 0]
# region_entities = entities[0, 0]
# print(len(region), len(region_entities))  # count of chunks in a region (1024 = 32*32)
# chunk = region[0, 0]  # 0->31
# chunk_entities = region_entities[0, 2]  # nothing at 0,0
# # mc.pretty(region)
# # mc.pretty(chunk)
# chunk["Level"].keys()

{   <Dimension.OVERWORLD: 0>: <Regions(937 regions)>,
    <Dimension.THE_NETHER: -1>: <Regions(4 regions)>,
    <Dimension.THE_END: 1>: <Regions(39 regions)>}
Count of regions: 937


In [6]:
mc.pretty(chunk_entities)

{
    Position: [I; 0, 2], 
    DataVersion: 2730, 
    Entities: [
        {
            Motion: [0.0d, 0.0d, 0.0d], 
            data: {}, 
            Invulnerable: 0b, 
            Air: 300s, 
            OnGround: 0b, 
            PortalCooldown: 0, 
            Rotation: [0.0f, 0.0f], 
            FallDistance: 0.0f, 
            Pos: [15.768591211414549d, 80.97479040622704d, 45.62760742379422d], 
            Fire: 0s, 
            id: "minecraft:marker", 
            UUID: [I; -25431429, -1147649921, -2132246914, 1342339230], 
            Tags: ["flammer"]
        }
    ]
}


In [3]:
def get_dimension_regions_and_entities(dimension: mc.Dimension) -> tuple[mc.Regions | dict, mc.Regions | dict]:
    """mcworld does not work for custom dimensions so to get them they will need to be temporarily renamed to replace THE_END or THE_NETHER"""
    world = mc.load(PATH_DREHMAL_level_dat)
    regions = world.regions[dimension]
    print("Count of regions:", len(regions))
    entities = world.entities[dimension]
    return regions, entities

In [4]:
def coordinate_to_region(block_coordinate: int) -> int:
    """Convert coordinates to region in the same dimension.

    :param block_coordinate: single x or z block coordinate
    :return: x or z region coordinate
    """
    return block_coordinate // 512

In [5]:
# Save to JSON
# Convert Numpy Arrays to useable
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)
    
def save_to_json_data(file_name: str, data: list[dict]):
    """Save the data to a JSON file in the SAVE_DIRECTORY.

    :param file_name: file name without .json
    :param data: list of dict data
    """
    save_path = SAVE_DIRECTORY / f"{file_name}.json"
    with open(save_path, 'w') as f:
        json.dump(data, f, cls=NumpyEncoder)

## Tile Entities

In [6]:
def tile_entity_has_items(tile_entity: nbtlib.tag.Compound) -> bool:
    """Does the tile entity have items in it, True or False.

    False if no 'Items' tag or no items stored.

    :param tile_entity: tile entity data
    :return: True if items in entity
    """
    if items := tile_entity.get("Items", None):
        if len(items) == 0:
            return False
        else:
            return True
    return False

def is_desired_tile_entity(tile_entity: nbtlib.tag.Compound) -> bool:
    """Is the tile entity wanted, True or False.

    Set values in ``TILE_ENTITIES`` sets above.

    :param tile_entity: tile entity data
    :return: True if wanted
    """
    entity_id = tile_entity["id"]
    if entity_id in TILE_ENTITIES_ITEMS:
        return tile_entity_has_items(tile_entity)
    elif entity_id in TILE_ENTITIES_OTHER:
        return True
    elif entity_id in TILE_ENTITIES_BOOK:
        if tile_entity.get("Book", None):
            # If there is a book in the lectern, it's good
            return True
        else:
            return False
    else:
        return False

def get_tile_entities_in_region(region: mc.RegionFile) -> list[dict]:
    tile_entities = []

    chunk_pos: mc.ChunkPos
    chunk: mc.RegionChunk
    for chunk_pos, chunk in region.items():
        if "Level" not in chunk:
            # possible to have empty chunks
            continue
        chunk_tile_entities = chunk["Level"]["TileEntities"]
        if len(chunk_tile_entities) == 0:
            continue

        # Determine if we want to keep the tile entity (has items, book, in set)
        for tile_entity in chunk_tile_entities:
            if is_desired_tile_entity(tile_entity):
                tile_entities.append(tile_entity)
    return tile_entities

# def get_all_tile_entities_in_region_range(regions: mc.Regions, dimension_name: str,
#                                           x_block_min: int, x_block_max: int,
#                                           z_block_min: int, z_block_max: int) -> list:
#     x_region_min = coordinate_to_region(x_block_min)
#     x_region_max = coordinate_to_region(x_block_max)
#     z_region_min = coordinate_to_region(z_block_min)
#     z_region_max = coordinate_to_region(z_block_max)
#    
#     all_tile_entities = []
#
#     for x in range(x_region_min, x_region_max + 1):
#         for z in range(z_region_min, z_region_max + 1):
#             if x % 5 == 0 and z % 5 == 0:
#                 print(f"On region {x}, {z}")
#             current_region = regions[x, z]
#             all_tile_entities.extend(get_tile_entities_in_region(current_region))
#
#     print(f"# of tile entities: {len(all_tile_entities)}")
#     return all_tile_entities

def get_all_tile_entities(regions: mc.Regions) -> list:
    all_tile_entities = []

    for i, (_, region_data) in enumerate(regions.items()):
        if i % 100 == 0:
            print(f"On region {i + 1}")
        current_region = region_data
        all_tile_entities.extend(get_tile_entities_in_region(current_region))

    print(f"# of tile entities: {len(all_tile_entities)}")
    return all_tile_entities

In [10]:
# overworld_tile_entities = get_all_tile_entities_in_region_range(
#     regions,
#     "overworld",
#     COORDINATES["Overworld"]["x_min"],
#     COORDINATES["Overworld"]["x_max"],
#     COORDINATES["Overworld"]["z_min"],
#     COORDINATES["Overworld"]["z_max"]
# )
overworld_tile_entities = get_all_tile_entities(regions)
print("done")

On region 1
On region 101
On region 201
On region 301
On region 401
On region 501
On region 601
On region 701
On region 801
On region 901
# of tile entities: 3680
done


In [20]:
save_to_json_data("overworld_tile_entities", overworld_tile_entities)

---
## Entities
Get actual entities including armor stands

In [7]:
def get_entities_in_region(region: mc.RegionFile) -> list[dict]:
    entities = []

    chunk_pos: mc.ChunkPos
    chunk: mc.RegionChunk
    for chunk_pos, chunk in region.items():
        if "Entities" not in chunk:
            # possible to have empty
            continue
        chunk_entities = chunk["Entities"]
        if len(chunk_entities) == 0:
            continue

        # for entity in chunk_entities:
        #     entities.append(entity)

        # Add all entities, no filtering here
        entities.extend(chunk_entities)
    return entities


def get_all_entities(entities: mc.Regions) -> list:
    all_entities = []

    for i, (_, region_data) in enumerate(entities.items()):
        if i % 50 == 0:
            print(f"On region {i + 1}")
        all_entities.extend(get_entities_in_region(region_data))

    print(f"# of entities: {len(all_entities)}")
    return all_entities

In [24]:
print(type(entities[-1954,-1954]))
mc.pretty(entities[-1954,-1954])

<class 'mcworldlib.anvil.RegionFile'>
{
    
}


In [10]:
overworld_entities = get_all_entities(entities)
print("done")

On region 1
On region 51
On region 101
On region 151
On region 201
On region 251
On region 301
On region 351
On region 401
On region 451
On region 501
On region 551
On region 601
On region 651
On region 701
On region 751
On region 801
# of entities: 26136
done


In [11]:
save_to_json_data("overworld_entities", overworld_entities)

---
## Get all data for a dimension

In [10]:
def get_all_data_for_dimension(mcworld_dim: mc.Dimension, dimension_override: str):
    regions, entities = get_dimension_regions_and_entities(mcworld_dim)

    t_start_tile_entities = time.perf_counter()
    tile_entities = get_all_tile_entities(regions)
    save_to_json_data(f"{dimension_override}_tile_entities", tile_entities)
    t_elapsed_tile = time.perf_counter()-t_start_tile_entities
    print(f"Tile Entities time: {dt.timedelta(seconds=t_elapsed_tile)}")

    t_start_entities = time.perf_counter()
    entities = get_all_entities(entities)
    save_to_json_data(f"{dimension_override}_entities", entities)
    t_elapsed_entity = time.perf_counter()-t_start_entities
    print(f"Entities time: {dt.timedelta(seconds=t_elapsed_entity)}")
    print(f"Done extracting for {dimension_override}")

In [14]:
get_all_data_for_dimension(mc.Dimension.THE_END, "lodahr")

Count of regions: 293
On region 1
On region 101
On region 201
# of tile entities: 813
Tile Entities time: 0:01:05.087496
On region 1
On region 51
On region 101
On region 151
On region 201
On region 251
On region 301
On region 351
On region 401
On region 451
# of entities: 5162
Entities time: 0:00:04.642780
Done extracting for lodahr
